## Теоретическая часть

#### 1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  В чем принципиальные отличия item-item рекомендаций от ALS?

User-User рекомендации основаны на анализе предпочтений групп пользователей со схожими интересами (при естественном предположении о консервативности пользовательских предпочтений). Item-Item рекомендации основаны на анализе взаимосвязей между объектами. В User-User рекомендации определяется сходство между пользователями и в качестве рекомендаций пользователю выдается n самых часто покупаемых товаров k наиболее похожими на него покупателями. Основная идея Item-Item рекомендации заключается
в группировке информационных единиц (товары, услуги, действия) имеющих сходные оценки пользователей. Рекомендации
вырабатываются по следующему принципу: пользователю оценившему объект X высоко будет предложен объект Y, который высоко оценили
другие пользователи, также высоко оценившие и объект X. У этих методов похожие алгоритмы выдачи рекомендаций.

В методе User-User, т.к. каждый пользователь взаимодействовал только с несколькими элементами, это делает метод довольно чувствительным к любым зарегистрированным взаимодействиям (высокая дисперсия). С другой стороны, мы получаем более персонализированные результаты. Этот метод чувствителен к холодному старту (новые объекты никому не рекомендуются) и метод не может ничего рекомендовать новым пользователям.

В методе Item-Item, т.к. многие пользователи взаимодействуют с элементом, поиск окрестностей менее чувствителен к отдельным взаимодействиям (более низкая дисперсия). Взаимодействия, исходящие от всех типов пользователей (даже пользователей, очень отличающихся от нашего эталонного пользователя), рассматриваются в рекомендации, что делает метод менее персонализированным. Таким образом, этот подход менее персонализирован, чем подход User-User, но более надежен. Этот метод так же чувствителен к холодному старту при этом рекомендации становятся тривиальными.

Функцию item_item_rec(interaction_matrix) можно использовать для user_user_rec добавляя в рекомендации вместе с товарами других пользователей и близкие к ним товары. Т.е. находим сначала похожих пользователей методом User-User , смотрим какие они товары покупали. Затем по товарам методом Item-Item находим похожие товары и рекомедуем их. 

Метод Item-Item дает рекомендации основанные на анализе взаимосвязей между объектами, используя неизменённую матрицу user_item_matrix. ALS - берёт user_item_matrix, приближенно факторизует её на две матрицы: матрицу факторов пользователей и матрицу факторов товаров. Рекомендации основываются на изменённой матрице user_item_matrix, которая получается как результат перемножения матриц факторов пользователей и факторов товаров.


#### 2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 

бинарные веса - купил/не купил,
максимальное число покупок, среднее число покупок, 
средняя сумма покупок, максимальная сумма покупок, нормированная сумма покупок ((x-min)/(max-min))

#### 3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)

Тип данных в матрице Cui - float. Пропуски в матрице заменяются нулями, что не вполне адекватно.
Если матрица Cui будет иметь высокий ранг, то сильного снижения размерности мы не получим.

ALS - это фактически множественная гребневая регрессия (регуляризация L2 приводит к тому, что ни один из коэффициентов не становится нулевым,  L1 регуляризаторы гарантируют отсутствие мультиколлинеарности и переобучения). Вычисление значений - линейно, т.к это умножение матриц.

## Практическая часть


In [39]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [41]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [42]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [43]:
# Для создания матрицы C_ui_quant_count используем функцию агрегации count, агрегацию производим по столбцу quantity  
c_ui_quant_count = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

c_ui_quant_count = c_ui_quant_count.astype(float) # необходимый тип матрицы для implicit


# Для создания матрицы C_ui_quant_max используем функцию агрегации max, агрегацию производим по столбцу quantity  
c_ui_quant_max = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='max', 
                                  fill_value=0
                                 )

c_ui_quant_max = c_ui_quant_max.astype(float) # необходимый тип матрицы для implicit


# Для создания матрицы C_ui_quant_mean используем функцию агрегации mean, агрегацию производим по столбцу quantity  
c_ui_quant_mean = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

c_ui_quant_mean = c_ui_quant_mean.astype(float) # необходимый тип матрицы для implicit

c_ui_quant_mean

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,8621.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Для создания матрицы C_ui_sales_max используем функцию агрегации max, агрегацию производим по столбцу sales_value  
c_ui_sales_max = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', 
                                  aggfunc='max', 
                                  fill_value=0
                                 )

c_ui_sales_max = c_ui_sales_max.astype(float) # необходимый тип матрицы для implicit


# Для создания матрицы C_ui_sales_mean используем функцию агрегации mean, агрегацию производим по столбцу sales_value  
c_ui_sales_mean = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', 
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

c_ui_sales_mean = c_ui_sales_mean.astype(float) # необходимый тип матрицы для implicit


c_ui_sales_mean

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,20.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
userids = c_ui_sales_mean.index.values
itemids = c_ui_sales_mean.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [48]:
%%time

# c_ui_quant_count, c_ui_quant_max, c_ui_quant_mean, c_ui_sales_max, c_ui_sales_mean

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(c_ui_quant_count).tocsr()

model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(sparse_user_item, show_progress=True)

def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res


Wall time: 1.27 s


In [52]:
%%time
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
    
#result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result

Wall time: 116 ms


,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [ ]:
for factor in [50, 100, 300]:
    model = ALS(factor, ...)

NameError: ignored

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [ ]:
!hh